Kode di bawah buat download datasetnya

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle/Rice Plant Detection"
%cd /content/gdrive/My Drive/Kaggle/Rice Plant Detection


/content/gdrive/My Drive/Kaggle/Rice Plant Detection


In [ ]:
! kaggle datasets download -d minhhuy2810/rice-diseases-image-dataset

In [ ]:
!unzip \*.zip  && rm *.zip

# MACHINE LEARNING STARTS HERE

In [4]:
Training_dir = os.path.join("/content/gdrive/MyDrive/Kaggle/Rice Plant Detection/RiceDiseaseDataset/train")
Testing_dir = os.path.join("/content/gdrive/MyDrive/Kaggle/Rice Plant Detection/RiceDiseaseDataset/validation")
brownspot_dir = os.path.join("/content/gdrive/MyDrive/Kaggle/Rice Plant Detection/RiceDiseaseDataset/train/BrownSpot")
healthy_dir = os.path.join("/content/gdrive/MyDrive/Kaggle/Rice Plant Detection/RiceDiseaseDataset/train/Healthy")
hispa_dir = os.path.join("/content/gdrive/MyDrive/Kaggle/Rice Plant Detection/RiceDiseaseDataset/train/Hispa")
leafblast_dir = os.path.join("/content/gdrive/MyDrive/Kaggle/Rice Plant Detection/RiceDiseaseDataset/train/LeafBlast")
brownspot_files = os.listdir(brownspot_dir)
healthy_files = os.listdir(healthy_dir)
hispa_files = os.listdir(hispa_dir)
leafblast_files = os.listdir(leafblast_dir)


In [ ]:
print('total training brown spot:', len(os.listdir(brownspot_dir)))
print('total training healthy:', len(os.listdir(healthy_dir)))
print('total training hispa:', len(os.listdir(hispa_dir)))
print('total training leaf blast:', len(os.listdir(leafblast_dir)))

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

pic_index = 2

next_brownspot = [os.path.join(brownspot_dir, fname) 
                for fname in brownspot_files[pic_index-2:pic_index]]
next_healthy = [os.path.join(healthy_dir, fname) 
                for fname in healthy_files[pic_index-2:pic_index]]
next_hispa = [os.path.join(hispa_dir, fname) 
                for fname in hispa_files[pic_index-2:pic_index]]
next_leafblast = [os.path.join(leafblast_dir, fname) 
                for fname in leafblast_files[pic_index-2:pic_index]]

for i, img_path in enumerate(next_brownspot+next_healthy+next_hispa+next_leafblast):
  img = mpimg.imread(img_path)
  plt.imshow(img)
  plt.axis('Off')
  plt.show()

In [7]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

In [ ]:
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=90,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	Training_dir,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=100
)
validation_generator = validation_datagen.flow_from_directory(
	Testing_dir,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=100
)

In [11]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

In [ ]:
model.summary()

model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit(train_generator, epochs=16, steps_per_epoch=1000, validation_data = validation_generator, verbose = 1, validation_steps=3)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()